In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # !pip install timm
# !pip install transformers
# !pip install swifter
# !pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'
# !pip install colorama
# !pip install wandb --upgrade
# !pip install iterative-stratification
# !pip install torchmetrics
# !pip install git+https://github.com/Atze00/MoViNet-pytorch.git
# !pip install "git+https://github.com/facebookresearch/pytorchvideo.git"

# # !pip install imageio-ffmpeg
# # !pip install -q mediapy

# !pip install datasets
# !pip install jiwer
# !apt-get install pv
# !pip install taco-box

# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import gc
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import time
import math
from datetime import datetime
from tqdm.notebook import tqdm
from typing import List, Dict, Any, Tuple
from types import SimpleNamespace
import swifter
import wandb
import torchsummary
# import mediapy as media

from PIL import Image, ImageDraw
import cv2

# augmentation
import albumentations as A

# model
# import timm
from movinets import MoViNet
from movinets.config import _C
from pytorchvideo.models.hub.slowfast import slowfast_16x8_r101_50_50
from torchvision.models.video import r3d_18, s3d, mvit_v2_s, r2plus1d_18

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.utils.data.sampler import Sampler
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, f1_score, classification_report, multilabel_confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold, train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# ------------------------------------------------------------------

from IPython.display import clear_output
from google.colab import output
output.enable_custom_widget_manager()
clear_output()
%matplotlib inline

from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

plt.rc('font', family='NanumBarunGothic')

In [ ]:
# !wandb login --relogin
# wandb.login()

In [ ]:
colab_v = "colabV9(mvits-stage1)"

"""
efficient b7 : 600
efficient b6 : 528
efficient b5 : 456
efficient b4 : 380
efficient b3 : 300
efficient b2 : 260
efficient b1 : 240
efficient b0 : 224
"""

cfg = SimpleNamespace()
cfg.model_name = "colabv9-stage1-mvits"
cfg.video_length = 16
cfg.img_h_size = 224
cfg.img_w_size = 224
cfg.lr = 2e-5
cfg.max_lr =1e-4
cfg.min_lr = 5e-6
cfg.T_max = 3
cfg.wd = 1e-2
cfg.eps = 1e-6
cfg.betas = (0.9, 0.999)
cfg.max_grad_norm = 1000
cfg.num_epochs = 30
cfg.patience = 7
cfg.accumulate_iter = 1
cfg.n_splits = 5
cfg.num_workers = 2
cfg.seed =  42
cfg.num_warmup_steps=0
cfg.train_batch_size = 8
cfg.valid_batch_size = 8
cfg.batch_scheduler = True

# slowfast cfg
cfg.sampling_rate = 2
cfg.frames_per_second = 10
cfg.slowfast_alpha = 4
cfg.image_selection = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,36,37,37,38,38,39,39,40,40,41,41,42,42,43,43,44,44,45,45,46,46,47,47,48,48,49,49]

cfg.date = datetime.now().strftime("%Y%m%d")
cfg.target_col = ['crash','ego','weather','timing']
cfg.stage_0_target_col = ['ego','weather','timing']
cfg.stage_1_target_col = ['crash_ego','weather','timing']

cfg.base_path = "/content/drive/MyDrive/dacon/Vision/car_crash_prediction/"
cfg.stage0_csv = cfg.base_path + "input/stage0_train.csv"
cfg.train_csv = cfg.base_path + "input/train.csv"
cfg.test_csv = cfg.base_path + "input/test.csv"
cfg.sample_submission_csv = cfg.base_path + "input/sample_submission.csv"



cfg.device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

In [ ]:
# !unzip /content/drive/MyDrive/dacon/Vision/car_crash_prediction/input/car_crash_prediction.zip -d /content/drive/MyDrive/dacon/Vision/car_crash_prediction/input/

In [ ]:
# if os.path.exists(f"/content/train/"):
#     shutil.rmtree("/content/train/")
# if os.path.exists(f"/content/test/"):
#     shutil.rmtree("/content/test/")
# !cp -r /content/drive/MyDrive/dacon/Vision/car_crash_prediction/input/train/ /content/train
# !cp -r /content/drive/MyDrive/dacon/Vision/car_crash_prediction/input/test/ /content/test

In [ ]:
def seed_everthing(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_everthing(cfg.seed)

pd.set_option('display.max_columns', 200) ## 모든 열 출력
pd.set_option('display.max_rows', 200) ## 모든 행 출력

# label 정보

- 13가지의 차량 충돌 상황 Class의 세부 정보
- crash : 차량 충돌 여부 (No/Yes)
- ego-Involve : 본인 차량의 충돌 사고 연류 여부 (No/Yes)
- weather : 날씨 상황 (Normal/Snowy/Rainy)
- timing : 낮과 밤 (Day/Night)
- ego-Involve, weather, timing의 정보는 '차량 충돌 사고'가 일어난 경우에만 분석

![image.png](https://dacon.s3.ap-northeast-2.amazonaws.com/competition/236064/editor-image/1675581601829146.jpeg)

- train.ego.value_counts() # 491(나와 충돌 1) : 424(상대 0)
- train.weather.value_counts() # 0(노멀) - 716, 1(눈) - 129, 2(비) - 70
- train.timing.value_counts() # 0 (낮)- 808, 1(밤) - 107



In [ ]:
def transform_label(x):
    
    if np.array_equal(x,np.array([1,0,0])):
        return 1
    elif np.array_equal(x,np.array([1,0,1])):
        return 2
    elif np.array_equal(x,np.array([1,1,0])):
        return 3
    elif np.array_equal(x,np.array([1,1,1])):
        return 4
    elif np.array_equal(x,np.array([1,2,0])):
        return 5
    elif np.array_equal(x,np.array([1,2,1])):
        return 6
    elif np.array_equal(x,np.array([0,0,0])):
        return 7
    elif np.array_equal(x,np.array([0,0,1])):
        return 8
    elif np.array_equal(x,np.array([0,1,0])):
        return 9
    elif np.array_equal(x,np.array([0,1,1])):
        return 10
    elif np.array_equal(x,np.array([0,2,0])):
        return 11
    elif np.array_equal(x,np.array([0,2,1])):
        return 12

def preprocessing_label(x):
    if x[0] == 0:
        return 0
    if np.array_equal(x,np.array([2,0,0])):
        return 1
    elif np.array_equal(x,np.array([2,0,1])):
        return 2
    elif np.array_equal(x,np.array([2,1,0])):
        return 3
    elif np.array_equal(x,np.array([2,1,1])):
        return 4
    elif np.array_equal(x,np.array([2,2,0])):
        return 5
    elif np.array_equal(x,np.array([2,2,1])):
        return 6
    elif np.array_equal(x,np.array([1,0,0])):
        return 7
    elif np.array_equal(x,np.array([1,0,1])):
        return 8
    elif np.array_equal(x,np.array([1,1,0])):
        return 9
    elif np.array_equal(x,np.array([1,1,1])):
        return 10
    elif np.array_equal(x,np.array([1,2,0])):
        return 11
    elif np.array_equal(x,np.array([1,2,1])):
        return 12



In [ ]:
train = pd.read_csv(cfg.train_csv)
test = pd.read_csv(cfg.test_csv)
sample_submission = pd.read_csv(cfg.sample_submission_csv)
stage0_train = pd.read_csv(cfg.stage0_csv)

In [ ]:
stage0_train['crash_ego'] = stage0_train['label'].swifter.apply(lambda x : 0 if x == 0 else (1 if x in [7,8,9,10,11,12] else 2))
stage0_train.drop('ego', axis=1, inplace=True)
test['video_path'] = test["video_path"].swifter.apply(lambda x : ''.join(['/content/', "/".join(x.split("/")[1:])]))

Pandas Apply:   0%|          | 0/2692 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1800 [00:00<?, ?it/s]

In [ ]:
# 비디오 체크

# label 0
# view_df = train[train['label']==0].sort_values('sample_id')
# for i in range(20, 100):
#     print(f"step {i}/{len(view_df)}")
#     video_path = view_df['video_path'].iloc[i]
#     id = view_df['sample_id'].iloc[i]
#     ego = view_df['ego'].iloc[i]
#     weather = view_df['weather'].iloc[i]
#     timing = view_df['timing'].iloc[i]
#     label = view_df['label'].iloc[i]

#     video = media.read_video(video_path)
#     print(f'Shape is (num_images, height, width, num_channels) = {video.shape}.')
#     print(f'Framerate is {video.metadata.fps} frames/s.')
#     print(f'video id : {id} video ego : {ego}, video weather : {weather}, video timing : {timing}, video label : {label}')
#     video = media.resize_video(video, np.array(video.shape[1:3]) // 4)
#     media.show_video(video, fps=10, codec='gif')

In [ ]:
def get_train_transforms():
    return A.Compose([
        A.Resize(cfg.img_h_size,cfg.img_w_size, interpolation=cv2.INTER_AREA, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.03, rotate_limit=0.03, p=0.3, border_mode=cv2.BORDER_REFLECT),
            # A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.2),
            # A.CLAHE(p=0.2),
            A.IAASharpen(alpha=(0.2,0.5), lightness=(0.2,0.5),p=0.3),
            A.CoarseDropout(max_holes=6, max_height=16, max_width=16, p=0.3),
            # A.ISONoise(p=0.2),
            # A.AdvancedBlur(p=0.2)
        ],p=0.3),

        A.Normalize()
    ], p=1.0, additional_targets={f"image{i}":"image" for i in range(1, 17)})

def get_valid_transforms():
    return A.Compose([
        A.Resize(cfg.img_h_size,cfg.img_w_size, interpolation=cv2.INTER_AREA, p=1.0),
        A.Normalize()
    ],p=1.0,  additional_targets={f"image{i}":"image" for i in range(1, 17)})

In [ ]:
def subplot_img(sub_plt, x, i, prefix):
    """
    Plot one frame of a 4D video tensor.
    Removes axes and prints Frame # for aesthetic peace of mind
    Parameters
    ----------
    sub_plt : matplotlib.figure.Figure
        One subplot of `plt.subplots`
    x : 4D torch.Tensor or np.array
        Must be of shape (T, H, W, C) (T = num frames)
    i : int
        Frame no. that is to be plotted
    """
    sub_plt.imshow(x[i])
    sub_plt.set_title(f'{prefix}Frame #{i+1}')
    sub_plt.axis('off')

import matplotlib.pyplot as plt

def plot_adjacent(x, prefix='', i=10, j=13, figsize=(30,24)):
    """
    Plot 2 frames of a 4D video tensor/array side by side
    Parameters
    ----------
    x : 4D torch.Tensor or np.array
        Must be of shape (T, H, W, C) (T = num frames)
    i1, i2 : int
        Indices of the frames to be plotted
    """

    f, plots = plt.subplots(1, 6, figsize=figsize)
    plots[0] = subplot_img(plots[0], x, i, prefix)
    plots[1] = subplot_img(plots[1], x, i+1, prefix)
    plots[2] = subplot_img(plots[2], x, i+2, prefix)
    plots[3] = subplot_img(plots[3], x, j, prefix)
    plots[4] = subplot_img(plots[4], x, j+1, prefix)
    plots[5] = subplot_img(plots[5], x, j+2, prefix)

    f.tight_layout()

In [ ]:
#  # 각 라벨에 대한 비디오
# # stage0_train = pd.read_csv(cfg.base_path + f"input/stage0_train.csv")
# # view_df = stage0_train[(stage0_train["label"]==0) & (stage0_train['weather']==2)].reset_index(drop=True).copy()
# view_df = stage0_train[stage0_train['sample_id']=='TRAIN_0541']
# for i in range(0, len(view_df)):
#     print(f"{i+1} / {len(view_df)}")
#     video_path = view_df['video_path'].iloc[i]
#     id = view_df['sample_id'].iloc[i]
#     label = view_df['label'].iloc[i]
#     ego = view_df['ego'].iloc[i]
#     weather = view_df['weather'].iloc[i]
#     timing = view_df['timing'].iloc[i]

#     video = media.read_video(video_path)
#     print(f'Shape is (num_images, height, width, num_channels) = {video.shape}.')
#     print(f'Framerate is {video.metadata.fps} frames/s.')
#     print(f'video id : {id} video label : {label} video ego : {ego}, video weather : {weather}, video timing : {timing}')
#     video = media.resize_video(video, np.array(video.shape[1:3]) // 4)
#     media.show_video(video, fps=10, codec='gif', )


In [ ]:
# def aug_video(path, train_tfms, valid_tfms):
#     seed = random.randint(0,99999)
#     aug_vid = []
#     origin_vid = []
#     cap = cv2.VideoCapture(path)
#     frame_n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     for _ in range(frame_n):
#         _, img = cap.read()
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         origin_vid.append((valid_tfms(image = np.asarray(img)))['image'])
#         aug_vid.append((train_tfms(image = np.asarray(img)))['image'])
#     return np.stack(origin_vid[18::2]), np.stack(aug_vid[18::2])

# vid = train[train['weather']==1]['video_path'].iloc[3]
# origin_vid, aug_vid = aug_video(vid, get_train_transforms(), get_valid_transforms())

# plot_adjacent(origin_vid,   prefix='Original ')
# plot_adjacent(aug_vid, prefix='Augmented ')

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, df, transform, mode):
        self.df = df
        self.transform = transform
        self.mode = mode
        
        # slowfast
        # self.frame_transform = PackPathway()
        # self.num_frames = cfg.video_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        video_path = self.df['video_path'].iloc[idx]
        # 
        images = self._get_video(video_path)

        if self.mode != 'test':
            ret = self.transform(**images)
            
            # if random.random()>0.7:
            #     ret = self.color_transform(**ret)
            images = np.zeros((len(images), cfg.img_h_size, cfg.img_w_size, 3), dtype=np.float32)
           
            images[0, :, :, :] = ret["image"]
            for i in range(1, len(images)):
                images[i, :, :, :] = ret[f"image{i}"]


            # slowfast
            # images = torch.from_numpy(images)
            # images = self.frame_transform(images.permute(3,0,1,2))

            crash_ego = self.df['crash_ego'].iloc[idx]
            weather = self.df['weather'].iloc[idx]
            timing = self.df['timing'].iloc[idx]
            label = self.df['label'].iloc[idx]
            return {'img' :  torch.tensor(images, dtype=torch.float32).permute(3,0,1,2),
                    "crash_ego" : torch.tensor(crash_ego, dtype=torch.int64),
                    "weather" : torch.tensor(weather, dtype=torch.int64),
                    "timing" : torch.tensor(timing, dtype=torch.int64),
                    "label" : torch.tensor(label, dtype=torch.int64)}
        else:
            ret = self.transform(**images)
            images = np.zeros((len(images),  cfg.img_h_size, cfg.img_w_size, 3), dtype=np.float32)
            images[0, :, :, :] = ret["image"]
            for i in range(1, len(images)):
                images[i, :, :, :] = ret[f"image{i}"]

            # slowfast

            # images = torch.from_numpy(images)
            # images = self.frame_transform(images.permute(3,0,1,2))

            return {'img' : torch.tensor(images, dtype=torch.float32).permute(3,0,1,2)} # [b, c, t, h, w]


    def _get_video(self, path):
        imgs = []
        cap = cv2.VideoCapture(path)
        frame_n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        for _ in range(frame_n):
            _, img = cap.read()
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img) # h,w,3
        # imgs = np.array(imgs)
        # imgs = np.take(imgs, cfg.image_selection, axis=0)  # 10 - 50 51부터는 50부터 뒤로 36까지 총 64 f
        imgs = imgs[18::2].copy()
        ret = {f"image{i}":imgs[i] for i in range(1, len(imgs))}
        ret['image'] = imgs[0]
        return ret




-------------------------------------------------

# stage0 train data preprocessing or clean

# 조사 weather 0인 것

# weather 
- TRAIN_0162 # weather 0 -> 1 길에 눈이 꽤 있음
- TRAIN_0209 # weather 0 -> 1 눈이 내리는데 비처럼 보임
- TRAIN_1319 # weather 0 -> 1 눈이 내리지만 비와 차이가 없음(어려움)
- TRAIN_1466 # weather 0 -> 1 눈이 길가에 많음(이걸어케볼지...)
- TRAIN_0708 # weather 0 -> 1 길가에 눈이 많음
- TRAIN_0262 # weather 0 -> 1 눈이 더 많음

- TRAIN_0094 # weather 0 -> 2 비가 옴
- TRAIN_0116 # weather 0 -> 2 비가 옴
- TRAIN_0183 # weather 0 -> 2 비가 옴
- TRAIN_0256 # weather 0 -> 2 비가 내리고 있음
- TRAIN_0305 # weather 0 -> 2 비가 조금씩 내리기 시작
- TRAIN_0308 # weather 0 -> 2 비가 조금씩 내리기 시작
- TRAIN_0315 # weather 0 -> 2 비가 창문에 많이 묻음
- TRAIN_0326 # weather 0 -> 2 비가 조금씩 내리고 있음
- TRAIN_0345 # weather 0 -> 2 비가 많이 내림
- TRAIN_0348 # weather 0 -> 2 비가 내렸다가 그친 것 같지만 비에 가까움
- TRAIN_0368 # weather 0 -> 2 비?창문에 뭐가 많이 묻어 있음
- TRAIN_0370 # weather 0 -> 2 비가 내림
- TRAIN_0383 # weather 0 -> 2 비가 내림
- TRAIN_0405 # weather 0 -> 2 비가 아주 조금씩 내림, 길에 물기 많음
- TRAIN_0422 # weather 0 -> 2 비가 조금씩 내리는 중
- TRAIN_0438 # weather 0 -> 2 비가 창문에 많이 묻음
- TRAIN_0459 # weather 0 -> 2 비가 창문에 많음
- TRAIN_0477 # weather 0 -> 2 비가 조금씩 내림
- TRAIN_0534 # weather 0 -> 2 비가 내림
- TRAIN_0540 # weather 0 -> 2 비가 내림
- TRAIN_0547 # weather 0 -> 2 비가 내리는지는 확실하지 않지만 비가 내린 - 것으로 판단해야함
- TRAIN_0552 # weather 0 -> 2 비가 내림(눈인가?)
- TRAIN_0600 # weather 0 -> 2 비가 내림
- TRAIN_0764 # weather 0 -> 2 비가 내리기 시작
- TRAIN_0798 # weather 0 -> 2 창문에 비가 있음
- TRAIN_0806 # weather 0 -> 2 창문에 비가 있음
- TRAIN_0808 # weather 0 -> 2 비가 내리기 시작
- TRAIN_0826 # weather 0 -> 2 비가 내리기 시작
- TRAIN_0827 # weather 0 -> 2 비가 내린 것 같음
- TRAIN_0859 # weather 0 -> 2 비가 내리기 시작
- TRAIN_0992 # weather 0 -> 2 창문에 비가 많음
- TRAIN_1020 # weather 0 -> 2 비가 내리기 시작
- TRAIN_1029 # weather 0 -> 2 창문에 비가 있음 애매함
- TRAIN_1079 # weather 0 -> 2 비가 창문에 있음
- TRAIN_1109 # weather 0 -> 2 비가 조금씩 내리고 있음
- TRAIN_1129 # weather 0 -> 2 비가 조금씩 내리고 있음
- TRAIN_1150 # weather 0 -> 2 비가 내리고 있음
- TRAIN_1190 # weather 0 -> 2 비가 내림
- TRAIN_1269 # weather 0 -> 2 비가 내릴 것 같음
- TRAIN_1317 # weather 0 -> 2 비가 내림
- TRAIN_1349 # weather 0 -> 2 비가 내림
- TRAIN_1396 # weather 0 -> 2 비가 내림
- TRAIN_1405 # weather 0 -> 2 비가 내림
- TRAIN_1411 # weather 0 -> 2 비가 내림
- TRAIN_1562 # weather 0 -> 2 비가 창문에 있음
- TRAIN_1582 # weather 0 -> 2 비인지 눈인지 내림(길가에 눈이 많음)
- TRAIN_1730 # weather 0 -> 2 비가 내리기 시작
- TRAIN_1767 # weather 0 -> 2 비가 내리는 듯
- TRAIN_1831  # weather 0 -> 2 비 분위기
- TRAIN_1841 # weather 0 -> 2 비가 내림
- TRAIN_1871 # weather 0 -> 2 비가 오는 듯
- TRAIN_2000 # weather 0 -> 2 비가 옴
- TRAIN_2023 # weather 0 ->2 비가 옴
- TRAIN_2057 # weather 0 -> 2 비옴
- TRAIN_2087 # weather 0 -> 2 비옴
- TRAIN_2168 # weather 0 -> 2 비가 옴
- TRAIN_2179 # weather 0 -> 2 비가 옴
- TRAIN_2200 # weather 0 -> 2 비가 옴
- TRAIN_2244 # weather 0 -> 2 비가 옴
- TRAIN_2250 # weather 0 -> 2 비가 옴
- TRAIN_2309 # weather 0 -> 2 비가 옴
- TRAIN_2311 # weather 0 -> 2 비가 옴
- TRAIN_2384 # weather 0 -> 2 비가 옴
- TRAIN_2452 # weather 0 -> 2 비가 옴
- TRAIN_2456 # weather 0 -> 2 비가 옴
- TRAIN_2518 # weather 0 -> 2 비가 조금씩 옴
- TRAIN_2559 # weather 0 -> 2 비가 창문에 있음
- TRAIN_2601 # weather 0 -> 2 비가 조금씩 내림
- TRAIN_2649 # weather 0 -> 2 비가 조금씩 옴
- TRAIN_2660 # weather 0 -> 2 비(눈)가 조금씩 옴

<br>

------------------------------------------------------------------

<br>

# timing
- TRAIN_0184 # timing 0 -> 1 밤인지 낮인지 모르겠음
- TRAIN_1183 # timing 0 -> 1 저녁에 가까움
- TRAIN_1632 # timing 0 -> 1 밤인데 0
- TRAIN_1798 # timing 0 -> 1 밤임
- TRAIN_1821 # timing 0 -> 1 밤에 가까움
- TRAIN_2398 # timing 0 -> 1 밤에 더 가까움
- TRAIN_2471 # timing 0 -> 1  밤인데 시야가 짧음
- TRAIN_2501 # timing 0 -> 1 밤에 더 가까움
- TRAIN_2583 # timing 0 -> 1 밤임
- TRAIN_2593 # timing 0 -> 1 밤에 더 가까운듯?
- TRAIN_0541 # timing 0 -> 1 밤에 가까움

- TRAIN_1206 # timing 1 -> 0 낮인데 그늘진 곳
- TRAIN_1715 # timing 1 -> 0 낮인데 밤
- TRAIN_1841 # timing 1 -> 0 낮임
- TRAIN_2113 # timing 1 -> 0 낮인데 그늘진 곳인듯
- TRAIN_2257 # timing 1 -> 0 낮임
- TRAIN_2478 # timing 1 -> 0 도로의 밑이라 낮인데 그늘짐
- TRAIN_2494 # timing 1 -> 0 낮인데 차들이 헤드라이트를 켜져있음
- TRAIN_2572 # timing 1 -> 0 낮임
- TRAIN_2639 # timing 1 -> 0 낮에 더 가까움

<br>

# 삭제 의심
- TRAIN_0679 # 주차장
- TRAIN_1526 # timing 1 -> 0 지하도로 낮인데(삭제?)
- TRAIN_0104 # 화면이 검은색(삭제)
- TRAIN_0124 # 차 안 영상 (삭제)
- TRAIN_0146 # 카메라가 떨어짐 (삭제?)
- TRAIN_0161 # 카메라가 떨어짐 (삭제?) 
- TRAIN_0182 # 카메라가 떨어져 있음 (삭제?)
- TRAIN_0187 # 카메라가 고정되어 있지 않음 (삭제?)
- TRAIN_0293 # 길가에 눈이 많음 (중복같음 삭제)
- TRAIN_0304 # 중복같음(삭제)
- TRAIN_0330 # 카메라 각도가 하늘 ( 삭제)
- TRAIN_0387 # 카메라가 떨어짐(삭제)
- TRAIN_0390 # 카메라 각도가 너무 위(삭제)
- TRAIN_0406 # 창문이 지저분 (삭제)
- TRAIN_0417 # 카메라가 많이 흔들림 (삭제)
- TRAIN_0437 # 카메라 떨어짐 (삭제)
- TRAIN_0469 # 창문이 너무 지저분 (삭제)
- TRAIN_0516 # 비인지 모르겠음 창문이 지저분(삭제)
- TRAIN_0525 # 비인지 모르겠음 창문이 지저분 (삭제)
- TRAIN_0607 # 카메라가 떨어짐 (삭제)
- TRAIN_0825 # 세차장(삭제?)
- TRAIN_0961 # 세차장(삭제?)
- TRAIN_0982 # 마지막에 카메라가 떨어짐
- TRAIN_1019 # 창문이 지저분 (삭제)
- TRAIN_1084 # 주차장(낮과 밤을 구별할 수 없음)
- TRAIN_1099 # 영상 질이 안좋음 (삭제)
- TRAIN_1113 # 영상이 검은색(삭제)
- TRAIN_1137 # 창문이 지저분, 창문에 비가 있는지 확인하기 어려움(삭제)
- TRAIN_1155 # 비가 내리는지 애매함(삭제)
- TRAIN_1184 # 카메라가 떨어져 있음 (삭제)
- TRAIN_1207 # 카메라가 떨어져 있음 (삭제)
- TRAIN_1220 # 지하도(저녁인지 낮인지 구분안됨, 삭제)
- TRAIN_1226 # 창문이 너무 지저분 (삭제)
- TRAIN_1304 # 판단이 어려움(삭제)
- TRAIN_1318 # 카메라가 안쪽임(삭제)
- TRAIN_1361 # 창문이 더러움(삭제)
- TRAIN_1417 # 길에 눈이 많은데 이를 어찌할지 모르겠음
- TRAIN_1421 # 주차장(삭제)
- TRAIN_1475 # 주차장
- TRAIN_1547 # 길에 눈이 많은데 이를 어찌할지 모르겠음
- TRAIN_1565 # 카메라가 떨어져 있음
- TRAIN_1612 # 주차장
- TRAIN_1680 # 창문이 지저분 (삭제)
- TRAIN_1763 # 화질이 너무 안좋음 (삭제)
- TRAIN_1773 # label 0에도 label 1에도 있음(삭제)
- TRAIN_1897 # 비? 눈? 애매함(삭제)
- TRAIN_1903 # 지하도로
- TRAIN_1919 # 카메라 떨어짐(삭제)
- TRAIN_1927 # 창문에 비가 있지만 애매함(삭제?)
- TRAIN_1934 # 삭제
- TRAIN_2006 # 비가 오는지 애매함(삭제)
- TRAIN_2044 # 화질이 안좋음(삭제)
- TRAIN_2156 # 카메라가 많이 흔들림 (삭제)
- TRAIN_2188 # label 1인듯? (삭제)
- TRAIN_2271 # 카메라가 떨어짐 (삭제)
- TRAIN_2326 # timing 1 -> 0 낮에 더 가까운 듯?(삭제?)
- TRAIN_2385 # 화질이 안좋음 (삭제)
- TRAIN_2427 # 카메라가 떨어짐 (삭제), weather 0 -> 2 비가 옴
- TRAIN_2469 # timing 1 -> 0 지하도로 영상 끝에 나오는.. (삭제?)
- TRAIN_2472 # 카메라가 떨어짐 (삭제)
- TRAIN_2493 # weather 0 -> 2 비가 창문에 있음 애매함(삭제?)
- TRAIN_2508 # 세차장 (삭제)
- TRAIN_2511 # 비가 오는지 모르겠음, 창문이 지저분 (삭제)
- TRAIN_2524 # 창문이 지저분 (삭제)
- TRAIN_2587 # 카메라가 많이 흔들림(삭제)
- TRAIN_2678 # 화면이 거의 검은색(삭제)
- TRAIN_2695 # 세차장(삭제)
- TRAIN_1870 # 사고날뻔 label 0 -> 1 내가 회피함, weather 0 -> 2 - 창문에 뭐가 많음(비인지 모르겠음) (삭제)

<br>

# --------------------------------------------------------------------------------

<br>

# label 0 , weather 1인 것
# weather
- TRAIN_0033 # weather 1 -> 0 눈이 도로에 있는데 밝은 날 - - TRAIN_0418 
- TRAIN_0041  # weather 1 -> 0 눈에 도로가에 있지만 밝은 날
- TRAIN_0079 # weather 1 -> 0 도로에 눈이 있지만 밝은 날
- TRAIN_0098 # weather 1 -> 0 도로에 눈이 있지만 밝은 날
- TRAIN_0137 # weather 1 -> 0 도로가에 눈이 있지만 밝은 날
- TRAIN_0160 # weather 1 -> 0 일반 밝은 날
- TRAIN_0203 # weather 1 -> 0 일반 밝은 날
- TRAIN_0219 # weather 1 -> 0 도로가에 눈이 있지만 밝은 날
- TRAIN_0226 # weather 1 -> 0 도로가에 눈이 있지만 밝은 날
- TRAIN_0230 # weather 1 -> 0 일반 날
- TRAIN_0233 # weather 1 -> 0 도로가에 눈이 많은데 조금  흐린날...
- TRAIN_0259 # weather 1 -> 0 도로가에 눈이 많은데 밝은 날
- TRAIN_0270 # weather 1 -> 0 도로가에 눈이 있는데 밝은 날
- TRAIN_0418  # weather 1 -> 0 눈이 도로가에 있는데 밝은날
- TRAIN_0575 # weather 1 -> 0 일반 날
- TRAIN_0629 # weather 1 -> 0 흐린데 snow는 아닌듯
- TRAIN_1268  # weather 1 -> 0 일반에 가까움
- TRAIN_1517 # weather 1 -> 0 일반에 더 가까움
- TRAIN_1595 # weather 1 -> 0 일반에 가까움
- TRAIN_1777  # weather 1 -> 0 완전 일반
- TRAIN_2134  # weather 1 -> 0 일반에 가까움
- TRAIN_2605 # weather 1 -> 0 일반에 가까움

- TRAIN_0063 # weather 1 -> 2 비인지 눈인지가 옴
- TRAIN_1286 # weather 1 -> 2 비가 내림(눈인지 모르겠음)
- TRAIN_1306 # weather 1 -> 2 눈이 도로가에 많은데 비가 내림
- TRAIN_2343  # weather 1 -> 2 눈이 많은데 비가 내림
- TRAIN_2408 # weather 1 -> 2 눈이 많은데 비가 내림
- TRAIN_2448  # weather 1 - > 2 비가 내림

# timing

- TRAIN_1247  # timing 0 -> 1 밤에 가까움
- TRAIN_1509 # timing 0 -> 1 밤에 가까움
- TRAIN_2086  # timing 0 -> 1 밤에 가까움

- TRAIN_0818 # timing 1 -> 0 낮

# 삭제 의심
- TRAIN_0471  # 카메라 떨어짐 (삭제)
- TRAIN_2292  # 화면이 검은 (삭제)

<br>

# ---------------------------------------------------------------------------------------------

<br>

# label 0 , weather 2인 것
# weather 
- TRAIN_0573 # weathe 2 -> 0 흐린 날
- TRAIN_0998 # weather 2 -> 0 흐린날
- TRAIN_1104  # weather 2 -> 0 일반 날
- TRAIN_1722 # weather 2 -> 0 비가 오지 않음
- TRAIN_1768 # weather 2 -> 0 흐린 날
- TRAIN_1882  # weather 2 -> 0 일반 밤
- TRAIN_2097 # weather 2 -> 0 일반 밤
- TRAIN_2517  # weather 2 -> 0 흐린 날

- TRAIN_0944  # weather 2 -> 1(0) 밝은 날, 도로가에 눈이 있음

<br>

# timing
- TRAIN_0650 # timing 0 -> 1 밤
- TRAIN_1242 # timing 0 -> 1 밤
- TRAIN_0823  # timing 1 -> 0 낮
- TRAIN_0898  # timing 1 -> 0 낮에 가까움
- TRAIN_1834  # timing 1 -> 0 낮에 가까움

<br>

# 삭제 의심
- TRAIN_1499 # 카메라 떨어짐 (삭제)






#label ==0, weather == 0, weather 0 -> 1 
- ["TRAIN_0162","TRAIN_0209","TRAIN_1319","TRAIN_1466","TRAIN_0708", "TRAIN_0262"]

# weather 0 -> 2
-  ["TRAIN_0094","TRAIN_0116","TRAIN_0183","TRAIN_0256","TRAIN_0305","TRAIN_0308","TRAIN_0315","TRAIN_0326","TRAIN_0345","TRAIN_0348","TRAIN_0368","TRAIN_0370","TRAIN_0383","TRAIN_0405","TRAIN_0422","TRAIN_0438","TRAIN_0459","TRAIN_0477","TRAIN_0534","TRAIN_0540","TRAIN_0547","TRAIN_0552","TRAIN_0600","TRAIN_0764","TRAIN_0798","TRAIN_0806","TRAIN_0808","TRAIN_0826","TRAIN_0827","TRAIN_0859","TRAIN_0992","TRAIN_1020","TRAIN_1029","TRAIN_1079","TRAIN_1109","TRAIN_1129","TRAIN_1150","TRAIN_1190","TRAIN_1269","TRAIN_1317","TRAIN_1349","TRAIN_1396","TRAIN_1405","TRAIN_1411","TRAIN_1562","TRAIN_1582","TRAIN_1730","TRAIN_1767","TRAIN_1831","TRAIN_1841","TRAIN_1871","TRAIN_2000","TRAIN_2023","TRAIN_2057","TRAIN_2087","TRAIN_2168","TRAIN_2179","TRAIN_2200","TRAIN_2244","TRAIN_2250","TRAIN_2309","TRAIN_2311","TRAIN_2384","TRAIN_2452","TRAIN_2456","TRAIN_2518","TRAIN_2559","TRAIN_2601","TRAIN_2649","TRAIN_2660"]

# timing 
# timing 0 -> 1
-  ["TRAIN_0184","TRAIN_1183","TRAIN_1632","TRAIN_1798","TRAIN_1821","TRAIN_2398","TRAIN_2471","TRAIN_2501","TRAIN_2583","TRAIN_2593","TRAIN_0541"]

# timing 1 -> 0
-  ["TRAIN_1206","TRAIN_1715","TRAIN_1841","TRAIN_2113","TRAIN_2257","TRAIN_2478","TRAIN_2494","TRAIN_2572","TRAIN_2639"]

# 삭제 의심
-  ["TRAIN_1526","TRAIN_0104","TRAIN_0124","TRAIN_0146","TRAIN_0161","TRAIN_0182","TRAIN_0187","TRAIN_0293","TRAIN_0304","TRAIN_0330","TRAIN_0387","TRAIN_0390","TRAIN_0406","TRAIN_0417","TRAIN_0437","TRAIN_0469","TRAIN_0516","TRAIN_0525","TRAIN_0607","TRAIN_0825","TRAIN_0961","TRAIN_0982","TRAIN_1019","TRAIN_1084","TRAIN_1099","TRAIN_1113","TRAIN_1137","TRAIN_1155","TRAIN_1184","TRAIN_1207","TRAIN_1220","TRAIN_1226","TRAIN_1304","TRAIN_1318","TRAIN_1361","TRAIN_1417","TRAIN_1421","TRAIN_1475","TRAIN_1547","TRAIN_1565","TRAIN_1612","TRAIN_1680","TRAIN_1763","TRAIN_1773","TRAIN_1897","TRAIN_1903","TRAIN_1919","TRAIN_1927","TRAIN_1934","TRAIN_2006","TRAIN_2044","TRAIN_2156","TRAIN_2188","TRAIN_2271","TRAIN_2326","TRAIN_2385","TRAIN_2427","TRAIN_2469","TRAIN_2472","TRAIN_2493","TRAIN_2508","TRAIN_2511","TRAIN_2524","TRAIN_2587","TRAIN_2678","TRAIN_2695","TRAIN_1870"]

# -------------------------------------------------------------------------

# weather 1
# weather 1 -> 0
-  ["TRAIN_0033","TRAIN_0041","TRAIN_0079","TRAIN_0098","TRAIN_0137","TRAIN_0160","TRAIN_0203","TRAIN_0219","TRAIN_0226","TRAIN_0230","TRAIN_0233","TRAIN_0259","TRAIN_0270","TRAIN_0418","TRAIN_0575","TRAIN_0629","TRAIN_1268","TRAIN_1517","TRAIN_1595","TRAIN_1777","TRAIN_2134","TRAIN_2605"]

# weather 1 -> 2
-  ["TRAIN_0063","TRAIN_1286","TRAIN_1306","TRAIN_2343","TRAIN_2408","TRAIN_2448"]

# timing 
# timing 0 -> 1
- ["TRAIN_1247","TRAIN_1509","TRAIN_2086"]
# timing 1 -> 0
# ["TRAIN_0818"] 

# 삭제 의심
- ["TRAIN_0471","TRAIN_2292"]

# ---------------------------------------------------------------------------------------------
# weather 2
# weather 2 -> 0
- ["TRAIN_0573","TRAIN_0998","TRAIN_1104","TRAIN_1722","TRAIN_1768","TRAIN_1882","TRAIN_2097","TRAIN_2517"]

# weather 2 -> 1
- ["TRAIN_0944"]

# timing 0 -> 1
-  ["TRAIN_0650","TRAIN_1242"]
# timing 1 -> 0
-  ["TRAIN_0823","TRAIN_0898","TRAIN_1834"]

# 삭제 
-  ["TRAIN_1499"]







In [ ]:
# stage0 data clean

stage0_train.loc[stage0_train['sample_id'].isin(["TRAIN_0162","TRAIN_0209","TRAIN_1319","TRAIN_1466","TRAIN_0708", "TRAIN_0262"]), "weather" ] = 1
stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0094","TRAIN_0116","TRAIN_0183","TRAIN_0256","TRAIN_0305","TRAIN_0308","TRAIN_0315","TRAIN_0326","TRAIN_0345",
     "TRAIN_0348","TRAIN_0368","TRAIN_0370","TRAIN_0383","TRAIN_0405","TRAIN_0422","TRAIN_0438","TRAIN_0459","TRAIN_0477",
     "TRAIN_0534","TRAIN_0540","TRAIN_0547","TRAIN_0552","TRAIN_0600","TRAIN_0764","TRAIN_0798","TRAIN_0806","TRAIN_0808",
     "TRAIN_0826","TRAIN_0827","TRAIN_0859","TRAIN_0992","TRAIN_1020","TRAIN_1029","TRAIN_1079","TRAIN_1109","TRAIN_1129",
     "TRAIN_1150","TRAIN_1190","TRAIN_1269","TRAIN_1317","TRAIN_1349","TRAIN_1396","TRAIN_1405","TRAIN_1411","TRAIN_1562",
     "TRAIN_1582","TRAIN_1730","TRAIN_1767","TRAIN_1831","TRAIN_1841","TRAIN_1871","TRAIN_2000","TRAIN_2023","TRAIN_2057",
     "TRAIN_2087","TRAIN_2168","TRAIN_2179","TRAIN_2200","TRAIN_2244","TRAIN_2250","TRAIN_2309","TRAIN_2311","TRAIN_2384",
     "TRAIN_2452","TRAIN_2456","TRAIN_2518","TRAIN_2559","TRAIN_2601","TRAIN_2649","TRAIN_2660"]), "weather" ] = 2

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0184","TRAIN_1183","TRAIN_1632","TRAIN_1798","TRAIN_1821","TRAIN_2398","TRAIN_2471","TRAIN_2501","TRAIN_2583","TRAIN_2593","TRAIN_0541"]
), "timing" ] = 1
stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_1206","TRAIN_1715","TRAIN_1841","TRAIN_2113","TRAIN_2257","TRAIN_2478","TRAIN_2494","TRAIN_2572","TRAIN_2639"]
), "timing" ] = 0

stage0_train.drop(stage0_train[stage0_train['sample_id'].isin(
    ["TRAIN_1526","TRAIN_0104","TRAIN_0124","TRAIN_0146","TRAIN_0161","TRAIN_0182","TRAIN_0187","TRAIN_0293",
     "TRAIN_0304","TRAIN_0330","TRAIN_0387","TRAIN_0390","TRAIN_0406","TRAIN_0417","TRAIN_0437","TRAIN_0469",
     "TRAIN_0516","TRAIN_0525","TRAIN_0607","TRAIN_0825","TRAIN_0961","TRAIN_0982","TRAIN_1019","TRAIN_1084",
     "TRAIN_1099","TRAIN_1113","TRAIN_1137","TRAIN_1155","TRAIN_1184","TRAIN_1207","TRAIN_1220","TRAIN_1226",
     "TRAIN_1304","TRAIN_1318","TRAIN_1361","TRAIN_1417","TRAIN_1421","TRAIN_1475","TRAIN_1547","TRAIN_1565",
     "TRAIN_1612","TRAIN_1680","TRAIN_1763","TRAIN_1773","TRAIN_1897","TRAIN_1903","TRAIN_1919","TRAIN_1927",
     "TRAIN_1934","TRAIN_2006","TRAIN_2044","TRAIN_2156","TRAIN_2188","TRAIN_2271","TRAIN_2326","TRAIN_2385",
     "TRAIN_2427","TRAIN_2469","TRAIN_2472","TRAIN_2493","TRAIN_2508","TRAIN_2511","TRAIN_2524","TRAIN_2587",
     "TRAIN_2678","TRAIN_2695","TRAIN_1870"])].index, axis=0, inplace=True)

# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0033","TRAIN_0041","TRAIN_0079","TRAIN_0098","TRAIN_0137","TRAIN_0160","TRAIN_0203","TRAIN_0219",
     "TRAIN_0226","TRAIN_0230","TRAIN_0233","TRAIN_0259","TRAIN_0270","TRAIN_0418","TRAIN_0575","TRAIN_0629",
     "TRAIN_1268","TRAIN_1517","TRAIN_1595","TRAIN_1777","TRAIN_2134","TRAIN_2605"]
), "weather" ] = 0

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0063","TRAIN_1286","TRAIN_1306","TRAIN_2343","TRAIN_2408","TRAIN_2448"]
), "weather" ] = 2

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_1247","TRAIN_1509","TRAIN_2086"]
), "timing" ] = 1

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0818"] 
), "timing" ] = 0

stage0_train.drop(stage0_train[stage0_train['sample_id'].isin(
    ["TRAIN_0471","TRAIN_2292"])].index, axis=0, inplace=True)

# --------------------------------------------------------------------------------------------------------------------------

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0573","TRAIN_0998","TRAIN_1104","TRAIN_1722","TRAIN_1768","TRAIN_1882","TRAIN_2097","TRAIN_2517"]
), "weather" ] = 0

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0944"]
), "weather" ] = 1

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0650","TRAIN_1242"]
), "timing" ] = 1

stage0_train.loc[stage0_train['sample_id'].isin(
    ["TRAIN_0823","TRAIN_0898","TRAIN_1834"]
), "timing" ] = 0

stage0_train.drop(stage0_train[stage0_train['sample_id'].isin(
    ["TRAIN_1499"])].index, axis=0, inplace=True)

In [ ]:
stage0_train.crash_ego.value_counts()

0    1720
2     486
1     416
Name: crash_ego, dtype: int64

In [ ]:
stage0_train.weather.value_counts()

0.0    2099
2.0     281
1.0     242
Name: weather, dtype: int64

In [ ]:
stage0_train.timing.value_counts()

0.0    1718
1.0     904
Name: timing, dtype: int64

In [ ]:
# train size reduce
# stage0_train.drop('ego', axis=1, inplace=True)
stage0_train.drop(stage0_train[(stage0_train['crash_ego']==0) & (stage0_train['weather']==0) & (stage0_train['timing']==0)].sample(n=22).index, axis=0, inplace=True)

In [ ]:
skf = StratifiedKFold(n_splits=cfg.n_splits, shuffle=True, random_state=cfg.seed)
# mlkf = MultilabelStratifiedKFold(n_splits=cfg.n_splits, shuffle=True, random_state=cfg.seed)
# stage0_train = train[train['label']!=0].reset_index(drop=True).copy()
stage0_train.reset_index(drop=True,inplace=True)
for fold, (trn_idx, val_idx) in enumerate(skf.split(stage0_train, stage0_train['label'])):
    stage0_train.loc[val_idx,"kfold"] = fold

In [ ]:
def macro_score_function(real, pred):
    return f1_score(real, pred, average="macro")

class FocalLoss(nn.Module):
    def __init__(self, weight=None,  gamma=2, smooth=0.1, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight=weight
        self.gamma = gamma
        self.smooth = smooth
        self.reduction = reduction

    def forward(self, inputs, targets):
        
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, label_smoothing=self.smooth, reduction=self.reduction)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()

        return focal_loss
                       
class EarlyStopping():
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._score = 0
        self._loss = float('inf')
        self.patience  = patience
        self.verbose = verbose

    def loss_validate(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print(f'Training process is stopped early....')
                return True
        else:
            self._step = 0
            self._loss = loss

    def score_validate(self,score):
        if self._score > score:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print(f"Training process is stopped early....")
                return True
        else:
            self._step = 0
            self._score = score

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [ ]:
class StageZeroModel(nn.Module):
    def __init__(self):
        super(StageZeroModel,self).__init__()
        self.model = mvit_v2_s(weights="KINETICS400_V1")
        # self.model = s3d(weights="KINETICS400_V1")
        # self.model = MoViNet(_C.MODEL.MoViNetA1, causal = True, pretrained = True)
        # self.model=slowfast_16x8_r101_50_50(pretrained=True, dropout_rate=0.7)
        # self.model.blocks[6].proj = nn.Linear(self.model.blocks[6].proj.in_features, 6, bias=True)
        # self.model.classifier[3] = nn.Conv3d(2048, 600, (1,1,1))
        
        self.softmax = nn.Softmax()

        # crash_ego
        self.classifier1 = self.get_classifier(0.5, 3)
        # weather
        self.classifier2 = self.get_classifier(0.5, 3)
        # timing
        self.classifier3 = self.get_classifier(0.5, 2)
        
        self._normal_init_weights(self.classifier1)
        self._normal_init_weights(self.classifier2)
        self._normal_init_weights(self.classifier3)
    
        
    def get_classifier(self, drop_rate, target_col):
            return nn.Sequential(
                nn.Linear(400, 400),
                nn.BatchNorm1d(400),
                nn.ReLU(),
                nn.Dropout(drop_rate),
                nn.Linear(400, target_col)
            )

    def _normal_init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight)
            module.bias.data.fill_(0.01)
        elif isinstance(module, nn.BatchNorm2d):
            module.weight.data.fill_(1)
            module.bias.data.zero_()

    def forward(self,x):
        x = self.model(x)    # torch.Size([2, 1024])   
        x0 = self.softmax(self.classifier1(x))
        x1 = self.softmax(self.classifier2(x))
        x2 = self.softmax(self.classifier3(x))
    
        return x0, x1, x2

In [ ]:
def evaluate_step(model, crash_ego_criterion, weather_criterion,timing_criterion, fold, valid_loader, device, epoch):


    running_loss = 0.0
    running_crash_ego_acc = 0.0
    running_weather_acc = 0.0
    running_timing_acc = 0.0
    

    model_crash_ego_preds = []
    model_weather_preds = []
    model_timing_preds = []
    true_labels = []

    model.eval()

    # movinet
    # model.model.clean_activation_buffers()

    with torch.no_grad():
        for i, item in enumerate(valid_loader):
            for k, v in item.items():
                item[k] = v.to(device, non_blocking=True)
                # slowfast
                # if k != 'img':
                # imgs = [x.to(cfg.device, non_blocking=True) for x in item['img']]

            # with torch.cuda.amp.autocast():
            output1, output2, output3 = model(item['img'])
            
            # ego
            loss1 = crash_ego_criterion(output1, item['crash_ego'])
            # weather
            loss2 = weather_criterion(output2, item['weather'])
            # timing
            loss3 = timing_criterion(output3, item['timing'])
    
            loss = (loss1 + loss2 + loss3) / 3
            # loss = (loss1*0.4) + (loss2*0.3) + (loss3*0.3)

            running_loss += loss.item()
            
            pred1 = output1.argmax(1).cpu().numpy()
            pred2 = output2.argmax(1).cpu().numpy()
            pred3 = output3.argmax(1).cpu().numpy()
     
            
            crash_ego = item['crash_ego'].detach().cpu().numpy()
            weather = item['weather'].detach().cpu().numpy()
            timing = item['timing'].detach().cpu().numpy()
         

            running_loss += loss.item()

            # 3 target
            running_crash_ego_acc += macro_score_function(pred1,crash_ego)
            running_weather_acc += macro_score_function(pred2,weather)
            running_timing_acc += macro_score_function(pred3,timing)

            model_crash_ego_preds.extend(pred1.tolist())
            model_weather_preds.extend(pred2.tolist())
            model_timing_preds.extend(pred3.tolist())

            true_labels += item['label'].detach().cpu().tolist()

            # movinet
            # model.model.clean_activation_buffers()

            if (i+1) % 20 == 0:
                print(f'iter: {i+1},  Val_Loss: {running_loss / (i+1):.4f} | Val_crash_ego_Acc: {running_crash_ego_acc / (i+1):.4f} | Val_Weather_Acc: {running_weather_acc / (i+1):.4f} | Val_Timing_Acc: {running_timing_acc / (i+1):.4f}')
    
    model_preds = np.stack([np.array(model_crash_ego_preds), np.array(model_weather_preds), np.array(model_timing_preds)], axis=1)           
    model_preds = np.apply_along_axis(preprocessing_label, 1, model_preds)

    val_f1_macro = macro_score_function(model_preds, np.array(true_labels))

    wandb.log({"valid_loss": running_loss / len(valid_loader), "valid_crash_ego_acc" : running_crash_ego_acc / len(valid_loader), "valid_weather_acc" : running_weather_acc / len(valid_loader), \
               "valid_timing_acc" : running_timing_acc / len(valid_loader), "val_f1_macro" : val_f1_macro}, step=epoch)
    torch.cuda.empty_cache()
    gc.collect()

    return running_loss / len(valid_loader), running_crash_ego_acc / len(valid_loader), running_weather_acc / len(valid_loader), running_timing_acc / len(valid_loader), val_f1_macro

def train_step(model, crash_ego_criterion,weather_criterion,timing_criterion, train_loader, lr_scheduler, optimizer,device, scaler, epoch, fold, model_name):

    running_loss = 0.0
    running_crash_ego_acc = 0.0
    running_weather_acc = 0.0
    running_timing_acc = 0.0

    model.train()

    model_crash_ego_preds = []
    model_weather_preds = []
    model_timing_preds = []
    true_labels = []

    # movinet
    # model.model.clean_activation_buffers()
    for i, item in enumerate(train_loader):
        for k, v in item.items():
            item[k] = v.to(device, non_blocking=True)
            # if k != 'img':
            
            #slowfast
            # imgs = [x.to(cfg.device, non_blocking=True) for x in item['img']]
           
     
        with torch.cuda.amp.autocast(enabled=True):

            output1,output2,output3 = model(item['img'])
            # ego
            loss1 = crash_ego_criterion(output1, item['crash_ego'])
            # weather
            loss2 = weather_criterion(output2, item['weather'])
            # timing
            loss3 = timing_criterion(output3, item['timing'])
            

        loss = (loss1 + loss2 + loss3) / 3
        # loss = (loss1*0.4) + (loss2*0.3) + (loss3*0.3)
        loss = loss / cfg.accumulate_iter

        scaler.scale(loss).backward()
        
        pred1 = output1.argmax(1).cpu().numpy()
        pred2 = output2.argmax(1).cpu().numpy()
        pred3 = output3.argmax(1).cpu().numpy()
        
        
        crash_ego = item['crash_ego'].detach().cpu().numpy()
        weather = item['weather'].detach().cpu().numpy()
        timing = item['timing'].detach().cpu().numpy()

        running_loss += loss.item()

        # 3 target
        running_crash_ego_acc += macro_score_function(pred1,crash_ego)
        running_weather_acc += macro_score_function(pred2,weather)
        running_timing_acc += macro_score_function(pred3,timing)

        # label
        model_crash_ego_preds.extend(pred1.tolist())
        model_weather_preds.extend(pred2.tolist())
        model_timing_preds.extend(pred3.tolist())
        
        true_labels += item['label'].detach().cpu().tolist()

        # movinet
        # model.model.clean_activation_buffers()
     
        if ((i + 1) % cfg.accumulate_iter == 0) or (i + 1 == len(train_loader)):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_grad_norm) 
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            if cfg.batch_scheduler:
                lr_scheduler.step()   

        # running_loss += loss.item() * cfg.accumulate_iter
        running_loss += loss.item() * cfg.accumulate_iter

        if (i+1) % 30 == 0:
            print(f'iter: {i+1}, Train_Loss: {running_loss / (i+1):.4f} | Train_Crash_Ego_Acc: {running_crash_ego_acc / (i+1):.4f} | Train_Weather_Acc: {running_weather_acc / (i+1):.4f} | Train_Timing_Acc: {running_timing_acc / (i+1):.4f} | LR : {lr_scheduler.get_lr()[0]:.7f}')
    

    model_preds = np.stack([np.array(model_crash_ego_preds), np.array(model_weather_preds), np.array(model_timing_preds)], axis=1)           
    model_preds = np.apply_along_axis(preprocessing_label, 1, model_preds)

    train_f1_macro = macro_score_function(model_preds, np.array(true_labels))
   
    torch.cuda.empty_cache()
    gc.collect()
    wandb.log({"train_loss": running_loss / len(train_loader), "train_crash_ego_acc" : running_crash_ego_acc / len(train_loader), "train_weather_acc" : running_weather_acc / len(train_loader),\
               "train_timing_acc" : running_timing_acc / len(train_loader), "train_f1_macro": train_f1_macro}, step=epoch)
    return running_loss / len(train_loader),   running_crash_ego_acc / len(train_loader), running_weather_acc / len(train_loader), running_timing_acc / len(train_loader), train_f1_macro



def fit_model(model_name, device):

    for fold in range(cfg.n_splits):
        
        # if fold >=1 :
        #     continue

        wandb.init(project="Dacon(car_crash_multilabel_prediction_stage1)", entity="kyubokwi", config=cfg.__dict__,  group="fold_" + str(fold), allow_val_change=True)
        wandb.run.name = f'{colab_v}-{model_name}'
        # wandb.run.save()
        print(f"***Fold {fold} start***")

        best_valid_loss = 9999
        best_valid_acc = 0
       

        train_losses = []
        valid_losses = []
        train_accs = []
        valid_accs = []


        x_trn = stage0_train[stage0_train['kfold']!=fold].reset_index(drop=True)
        x_val = stage0_train[stage0_train['kfold']==fold].reset_index(drop=True)

        model = StageZeroModel()
        model.to(device, non_blocking=True)
        
        train_dataset = CustomDataset(x_trn , get_train_transforms(), 'train')
        valid_dataset = CustomDataset(x_val,  get_valid_transforms(), 'valid')
        
        # train_sampler = MultilabelBalancedRandomSampler(np.array(x_trn[cfg.stage_0_target_col].values))

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg.train_batch_size,
                                num_workers=cfg.num_workers, shuffle=True, pin_memory=True)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=cfg.valid_batch_size,
                                num_workers=cfg.num_workers, shuffle=False, pin_memory=True)

        early_stopping = EarlyStopping(patience=cfg.patience, verbose=1)
        
        # class weight
        crash_ego_sample_list = list(x_trn.crash_ego.value_counts())
        crash_ego_weights = [1 - (x / sum(crash_ego_sample_list)) for x in crash_ego_sample_list]
        crash_ego_weights = torch.FloatTensor(crash_ego_weights).to(device)

        weather_sample_list = list(x_trn.weather.value_counts())
        weather_weights = [1 - (x / sum(weather_sample_list)) for x in weather_sample_list]
        weather_weights = torch.FloatTensor(weather_weights).to(device)

        timing_sample_list = list(x_trn.timing.value_counts())
        timing_weights = [1 - (x / sum(timing_sample_list)) for x in timing_sample_list]
        timing_weights = torch.FloatTensor(timing_weights).to(device)

        crash_ego_criterion = FocalLoss(weight=crash_ego_weights, smooth=0.1).to(device)
        weather_criterion = FocalLoss(weight=weather_weights, smooth=0.1).to(device)
        timing_criterion = FocalLoss(weight=timing_weights, smooth=0.1).to(device)

        # ego_metric = Accuracy('multiclass', num_classes=2)
        # weather_metric = Accuracy('multiclass', num_classes=3)
        # timing_metric = Accuracy('multiclass', num_classes=2)

        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(
                nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]


        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=cfg.lr, eps = cfg.eps, weight_decay=cfg.wd)
    
        total_samples = len(train_loader)
        num_warmup_steps = int(total_samples * cfg.num_epochs * 0.05)
        num_training_steps = total_samples * cfg.num_epochs
        lr_scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=int(num_training_steps*1.7), num_cycles=0.5)

        scaler = torch.cuda.amp.GradScaler()
    
        wandb.watch(model, log="all", log_freq=10) 

        del train_dataset, valid_dataset
        gc.collect()

        for epoch in range(cfg.num_epochs):

            start_time = time.time()

            print(f' ** Epoch {epoch+1} start / lr : {lr_scheduler.optimizer.param_groups[0]["lr"]:.7f} **')


            train_loss, train_crash_ego_acc, train_weather_acc, train_timing_acc, train_f1_macro  = train_step(model, crash_ego_criterion,weather_criterion,timing_criterion, train_loader, lr_scheduler, optimizer,  device, scaler, epoch, fold, model_name)
            valid_loss, valid_crash_ego_acc, valid_weather_acc, valid_timing_acc, val_f1_macro = evaluate_step(model, crash_ego_criterion,weather_criterion,timing_criterion, fold, valid_loader, device, epoch)

            # train_acc = (train_ego_acc + train_weather_acc + train_timing_acc) / 3
            # valid_acc = (valid_ego_acc + valid_weather_acc + valid_timing_acc) / 3


            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
            train_accs.append(train_f1_macro)
            valid_accs.append(val_f1_macro)

            if val_f1_macro > best_valid_acc:
                print(f"{g_}Validation Acc Increased from {best_valid_acc} to {val_f1_macro}{sr_}")
                print('best valid acc achieved')
                best_valid_acc = val_f1_macro
                if not os.path.exists(cfg.base_path + f'/model/{model_name}'):
                    os.mkdir( cfg.base_path + f'/model/{model_name}')
                torch.save(model.state_dict(), cfg.base_path + f'/model/{model_name}/FOLD{fold}_{cfg.date}.pt')
                


            # if lr_scheduler:
            #     lr_scheduler.step()

            end_time = time.time()

            epoch_mins, epoch_secs = epoch_time(start_time, end_time)
            print(f'\n** Epoch {epoch+1} ended **')
            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.5f} | Train F1 Macro Acc: {train_f1_macro:0.4f} | Train Crash-Ego Acc: {train_crash_ego_acc:0.4f} | Train Weather Acc: {train_weather_acc:0.4f} | Train Timing Acc: {train_timing_acc:0.4f} \n')
            print(f'\tVal Loss: {valid_loss:.5f} | Val F1 Macro Acc: {val_f1_macro:0.4f} | Valid  Crash-Ego Acc: {valid_crash_ego_acc:0.4f} | Valid  Weather Acc: {valid_weather_acc:0.4f} | Valid Timing Acc: {valid_timing_acc:0.4f} \n\n')

        
            if early_stopping.score_validate(val_f1_macro):
                break

    torch.cuda.empty_cache()
    gc.collect()
    wandb.finish()

def inference():

    fold_preds = 0
    for fold in range(1):
        print(f"fold {fold+1}")
        
        model = StageZeroModel()
        model.to(cfg.device, non_blocking=True)
    
        model.load_state_dict(torch.load( cfg.base_path + f'model/{cfg.model_name}/FOLD{fold}_20230304.pt'))
        # model.load_state_dict(torch.load( cfg.base_path + f'/model/{cfg.model_name}/FOLD{fold}_{cfg.date}.pt'))
      
        model.eval()
        test_dataset = CustomDataset(test,  get_valid_transforms(), 'test')
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16,
                                num_workers=cfg.num_workers, shuffle=False, drop_last=False)
        
        
        print("loader")
        fold_crash_ego = 0
        fold_weather = 0
        fold_timing = 0
        crash_ego = []
        weather = []
        timing = []
        with torch.no_grad():
            for i, item in enumerate(test_loader):
                print(f"completed {i} / {len(test_loader)}")
                for k, v in item.items():
                    item[k] = v.to(cfg.device)
                # for k, v in item.items():
                    #slowfast
                    # if k != 'img':
                    #     item[k] = v.to(cfg.device, non_blocking=True)
                # imgs = [x.to(cfg.device, non_blocking=True) for x in item['img']]

                output1,output2,output3 = model(item['img'])
                pred1 = output1.cpu().numpy()
                pred2 = output2.cpu().numpy()
                pred3 = output3.cpu().numpy()
                crash_ego.extend(pred1)
                weather.extend(pred2)
                timing.extend(pred3)
            crash_ego = np.array(crash_ego)
            weather = np.array(weather)
            timing = np.array(timing)


            # oof
            # if not os.path.exists(cfg.base_path + f'output/{cfg.model_name}'):
            #         os.mkdir( cfg.base_path + f'output/{cfg.model_name}')
            # oof = pd.DataFrame(preds)
            # oof.to_csv(cfg.base_path + f'output/{cfg.model_name}/{cfg.model_name}_fold{fold}_oof_.csv', index=False)

        fold_crash_ego += crash_ego / 3
        fold_weather += weather / 3
        fold_timing += timing / 3
        # print(fold_crash_ego.shape)
    model_preds = np.stack([fold_crash_ego.argmax(1), fold_weather.argmax(1), fold_timing.argmax(1)], axis=1)           
    model_preds = np.apply_along_axis(preprocessing_label, 1, model_preds)
    print(model_preds.shape)
    
    
    print("submission")
    submission = pd.read_csv(cfg.sample_submission_csv)
    submission.loc[:,'label'] = model_preds.tolist()
    submission.to_csv(cfg.base_path + f'output/{cfg.model_name}_{cfg.date}_submission.csv', index=False)
 
    return submission

In [ ]:
fit_model(model_name=cfg.model_name, device=cfg.device)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kyubokwi. Use `wandb login --relogin` to force relogin


***Fold 0 start***
 ** Epoch 1 start / lr : 0.0000200 **
iter: 30, Train_Loss: 0.8823 | Train_Crash_Ego_Acc: 0.2646 | Train_Weather_Acc: 0.2106 | Train_Timing_Acc: 0.4047 | LR : 0.0000200
iter: 60, Train_Loss: 0.8873 | Train_Crash_Ego_Acc: 0.2861 | Train_Weather_Acc: 0.1850 | Train_Timing_Acc: 0.4428 | LR : 0.0000200
iter: 90, Train_Loss: 0.8710 | Train_Crash_Ego_Acc: 0.3062 | Train_Weather_Acc: 0.1982 | Train_Timing_Acc: 0.4984 | LR : 0.0000200
iter: 120, Train_Loss: 0.8585 | Train_Crash_Ego_Acc: 0.3262 | Train_Weather_Acc: 0.2017 | Train_Timing_Acc: 0.5476 | LR : 0.0000200
iter: 150, Train_Loss: 0.8500 | Train_Crash_Ego_Acc: 0.3464 | Train_Weather_Acc: 0.1960 | Train_Timing_Acc: 0.5973 | LR : 0.0000200
iter: 180, Train_Loss: 0.8411 | Train_Crash_Ego_Acc: 0.3515 | Train_Weather_Acc: 0.2012 | Train_Timing_Acc: 0.6303 | LR : 0.0000200
iter: 210, Train_Loss: 0.8370 | Train_Crash_Ego_Acc: 0.3531 | Train_Weather_Acc: 0.1968 | Train_Timing_Acc: 0.6592 | LR : 0.0000200
iter: 240, Train_Loss:

train_crash_ego_acc,▁▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇██▇███████████
train_f1_macro,▁▂▂▃▃▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇███▇
train_loss,█▆▆▅▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_timing_acc,▁▅▆▇▇▇▇▇▇▇▇▇█▇▇██▇████████████
train_weather_acc,▁▂▃▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████
val_f1_macro,▁▃▃▄▅▅▆▇▆▆██▇█████▇█▇▇▇▇█▇▇██▇
valid_crash_ego_acc,▁▆▆▆▇▇█▇█▇████████████████████
valid_loss,█▆▄▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
valid_timing_acc,▁▂▂▇▇▆▇█▇▆█▆▆▆▇▇▇█▇▆▇▇█▆▇▆▆█▇▇
valid_weather_acc,▁▄▅▅▆▆▇▇▇▇▇▇██▇█████▇█████████
train_crash_ego_acc,0.9641


***Fold 1 start***
 ** Epoch 1 start / lr : 0.0000200 **
iter: 30, Train_Loss: 0.9078 | Train_Crash_Ego_Acc: 0.2525 | Train_Weather_Acc: 0.2738 | Train_Timing_Acc: 0.4452 | LR : 0.0000200
iter: 60, Train_Loss: 0.8905 | Train_Crash_Ego_Acc: 0.2938 | Train_Weather_Acc: 0.2830 | Train_Timing_Acc: 0.4553 | LR : 0.0000200
iter: 90, Train_Loss: 0.8765 | Train_Crash_Ego_Acc: 0.2904 | Train_Weather_Acc: 0.2934 | Train_Timing_Acc: 0.5076 | LR : 0.0000200
iter: 120, Train_Loss: 0.8696 | Train_Crash_Ego_Acc: 0.2885 | Train_Weather_Acc: 0.2946 | Train_Timing_Acc: 0.5495 | LR : 0.0000200
iter: 150, Train_Loss: 0.8594 | Train_Crash_Ego_Acc: 0.3002 | Train_Weather_Acc: 0.3067 | Train_Timing_Acc: 0.5848 | LR : 0.0000200
iter: 180, Train_Loss: 0.8450 | Train_Crash_Ego_Acc: 0.3112 | Train_Weather_Acc: 0.3181 | Train_Timing_Acc: 0.6203 | LR : 0.0000200
iter: 210, Train_Loss: 0.8378 | Train_Crash_Ego_Acc: 0.3247 | Train_Weather_Acc: 0.3097 | Train_Timing_Acc: 0.6560 | LR : 0.0000200
iter: 240, Train_Loss:

train_crash_ego_acc,▁▂▄▅▆▆▆▆▇▇▇▇▇▇████████
train_f1_macro,▁▁▂▂▃▃▄▄▅▅▆▅▆▆▆▇▇▇████
train_loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
train_timing_acc,▁▆▇▇▇▇▇█▇█▇███████████
train_weather_acc,▁▂▂▂▃▄▄▄▅▅▆▆▆▇▇▇▇▇▇███
val_f1_macro,▁▂▂▃▄▄▅▆▆▆▆▆▇█▇▇▇█▇▇▇█
valid_crash_ego_acc,▁▃▅▆▇▆▇▇▇▇▇█▇█████████
valid_loss,█▇▅▄▄▃▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
valid_timing_acc,▅▅▁▆▅▆▆▇▇▆▇▇▇▇█▆▇█████
valid_weather_acc,▁▃▃▄▅▅▆▇▇▇█▆███▇██▇▇█▇
train_crash_ego_acc,0.93975


***Fold 2 start***
 ** Epoch 1 start / lr : 0.0000200 **
iter: 30, Train_Loss: 0.8815 | Train_Crash_Ego_Acc: 0.3042 | Train_Weather_Acc: 0.1764 | Train_Timing_Acc: 0.5111 | LR : 0.0000200
iter: 60, Train_Loss: 0.8789 | Train_Crash_Ego_Acc: 0.3052 | Train_Weather_Acc: 0.1627 | Train_Timing_Acc: 0.5698 | LR : 0.0000200
iter: 90, Train_Loss: 0.8613 | Train_Crash_Ego_Acc: 0.3127 | Train_Weather_Acc: 0.1605 | Train_Timing_Acc: 0.6272 | LR : 0.0000200
iter: 120, Train_Loss: 0.8423 | Train_Crash_Ego_Acc: 0.3275 | Train_Weather_Acc: 0.1762 | Train_Timing_Acc: 0.6886 | LR : 0.0000200
iter: 150, Train_Loss: 0.8325 | Train_Crash_Ego_Acc: 0.3327 | Train_Weather_Acc: 0.1886 | Train_Timing_Acc: 0.7279 | LR : 0.0000200
iter: 180, Train_Loss: 0.8188 | Train_Crash_Ego_Acc: 0.3393 | Train_Weather_Acc: 0.2034 | Train_Timing_Acc: 0.7497 | LR : 0.0000200
iter: 210, Train_Loss: 0.8108 | Train_Crash_Ego_Acc: 0.3516 | Train_Weather_Acc: 0.2096 | Train_Timing_Acc: 0.7701 | LR : 0.0000200
iter: 240, Train_Loss:

KeyboardInterrupt: ignored

In [ ]:
# result = inference()

fold
loader
completed 0 / 113
completed 1 / 113
completed 2 / 113
completed 3 / 113
completed 4 / 113
completed 5 / 113
completed 6 / 113
completed 7 / 113
completed 8 / 113
completed 9 / 113
completed 10 / 113
completed 11 / 113
completed 12 / 113
completed 13 / 113
completed 14 / 113
completed 15 / 113
completed 16 / 113
completed 17 / 113
completed 18 / 113
completed 19 / 113
completed 20 / 113
completed 21 / 113
completed 22 / 113
completed 23 / 113
completed 24 / 113
completed 25 / 113
completed 26 / 113
completed 27 / 113
completed 28 / 113
completed 29 / 113
completed 30 / 113
completed 31 / 113
completed 32 / 113
completed 33 / 113
completed 34 / 113
completed 35 / 113
completed 36 / 113
completed 37 / 113
completed 38 / 113
completed 39 / 113
completed 40 / 113
completed 41 / 113
completed 42 / 113
completed 43 / 113
completed 44 / 113
completed 45 / 113
completed 46 / 113
completed 47 / 113
completed 48 / 113
completed 49 / 113
completed 50 / 113
completed 51 / 113
completed 

In [ ]:
result

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0
